In [1]:
import pandas as pd
import numpy as np

In [2]:
gameInput = pd.read_csv("nhl-game-data/game.csv").dropna()
# goalieInput = pd.read_csv("nhl-game-data/game_goalie_stats.csv")
# skaterStatsInput = pd.read_csv("nhl-game-data/game_skater_stats.csv")
# teamsStatsInput = pd.read_csv("nhl-game-data/game_teams_stats.csv")
# teamInfoInput = pd.read_csv("nhl-game-data/team_info.csv")

In [3]:
gameInput

,game_id,season,type,date_time,date_time_GMT,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
0,2011030221,20112012,P,2012-04-29,2012-04-29T19:00:00Z,1,4,3,4,home win OT,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
1,2011030222,20112012,P,2012-05-01,2012-05-01T23:30:00Z,1,4,4,1,away win REG,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
2,2011030223,20112012,P,2012-05-03,2012-05-03T23:30:00Z,4,1,3,4,home win OT,left,Prudential Center,/api/v1/venues/null,America/New_York,-4,EDT
3,2011030224,20112012,P,2012-05-06,2012-05-06T23:30:00Z,4,1,2,4,home win REG,left,Prudential Center,/api/v1/venues/null,America/New_York,-4,EDT
4,2011030225,20112012,P,2012-05-08,2012-05-08T23:30:00Z,1,4,3,1,away win REG,right,Wells Fargo Center,/api/v1/venues/null,America/New_York,-4,EDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,2018030413,20182019,P,2019-06-02,2019-06-02T00:00:00Z,6,19,7,2,away win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT
11430,2018030414,20182019,P,2019-06-04,2019-06-04T00:00:00Z,6,19,2,4,home win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT
11431,2018030415,20182019,P,2019-06-07,2019-06-07T00:00:00Z,19,6,2,1,away win REG,left,TD Garden,/api/v1/venues/5085,America/New_York,-4,EDT
11432,2018030416,20182019,P,2019-06-10,2019-06-10T00:00:00Z,6,19,5,1,away win REG,left,Enterprise Center,/api/v1/venues/5076,America/Chicago,-5,CDT


In [6]:
# gameAwayValues = gameInput.loc[:,['away_team_id','date_time', 'away_goals','home_goals','season']] 
gameAwayValues = gameInput.loc[:,['away_team_id', 'date_time', 'away_goals', 'home_goals', 'season']].groupby(['away_team_id', 'season']) \
       .agg({'date_time':'size', 'away_goals':'mean', 'home_goals':'mean'}) \
       .rename(columns={'date_time':'count'}) \
       .reset_index()
gameHomeValues = gameInput.loc[:,['home_team_id', 'date_time', 'away_goals', 'home_goals', 'season']].groupby(['home_team_id', 'season']) \
       .agg({'date_time':'size', 'away_goals':'mean', 'home_goals':'mean'}) \
       .rename(columns={'date_time':'count'}) \
       .reset_index()
gameAwayValues.head(5)

,away_team_id,season,count,away_goals,home_goals
0,1,20102011,39,1.948718,2.538462
1,1,20112012,53,2.566038,2.528302
2,1,20122013,24,1.916667,2.916667
3,1,20132014,40,2.250000,2.825000
4,1,20142015,41,2.195122,3.073171


In [5]:
homeTeamStats = pd.DataFrame(columns=['home_team_id', 'season', 'scored', 'conceded'])
awayTeamStats = pd.DataFrame(columns=['away_team_id', 'season', 'scored', 'conceded'])
for index, row in gameAwayValues.iterrows():
    if not gameHomeValues.loc[(gameHomeValues['home_team_id'] == row['away_team_id']) & (gameHomeValues['season'] == row['season'])].empty:
        season = row['season']
        homeTeam = gameHomeValues.loc[(gameHomeValues['home_team_id'] == row['away_team_id']) & (gameHomeValues['season'] == row['season'])]
        homeScored = (homeTeam['home_goals'].values[0]*homeTeam['count'].values[0] + row['home_goals']*row['count'])/(homeTeam['count'].values[0] + row['count'])
        awayScored = (homeTeam['away_goals'].values[0]*homeTeam['count'].values[0] + row['away_goals']*row['count'])/(homeTeam['count'].values[0] + row['count'])
        d1 = {'away_team_id': row['away_team_id'], 'season': season, 'conceded': awayScored, 'scored': homeScored}
        d2 = {'home_team_id': homeTeam['home_team_id'].values[0], 'season': season, 'conceded': awayScored, 'scored': homeScored}
        homeTeamStats.at[index, :] = d2
        awayTeamStats.at[index, :] = d1
awayTeamStats.head()

,away_team_id,season,scored,conceded
0,1,2.0102e+07,2.4125,2.2125
1,1,2.0112e+07,2.68571,2.51429
2,1,2.0122e+07,2.83333,2.1875
3,1,2.0132e+07,2.69136,2.24691
4,1,2.0142e+07,2.64634,2.19512


In [17]:
dataSet1 = pd.DataFrame(columns=['game_id', 'season', 'date_time', 'home_team_id', '1_scored', '1_conceded', '1_scored_avg', '1_conceded_avg', 'away_team_id', '2_scored', '2_conceded', '2_scored_avg', '2_conceded_avg', 'total'])
# print(gameAwayValues.loc[(gameAwayValues['away_team_id'] == '4') & (gameAwayValues['season'] == '20102011')])
for index, row in gameInput.iterrows():
    season = row['season'] - 10001
    if not awayTeamStats.loc[(awayTeamStats['away_team_id'] == row['away_team_id']) & (awayTeamStats['season'] == season)].empty:
        if not homeTeamStats.loc[(homeTeamStats['home_team_id'] == row['home_team_id']) & (homeTeamStats['season'] == season)].empty:
            awayTeam = awayTeamStats.loc[(awayTeamStats['away_team_id'] == row['away_team_id']) & (awayTeamStats['season'] == row['season'])]
            homeTeam = homeTeamStats.loc[(homeTeamStats['home_team_id'] == row['home_team_id']) & (homeTeamStats['season'] == row['season'])]
            d = {'game_id': row['game_id'],\
                 'season': row['season'], \
                 'date_time': row['date_time'], \
                 'home_team_id': row['home_team_id'], \
                 '1_scored_avg': homeTeam['scored'].values[0], \
                 '1_scored': row["home_goals"], \
                 '1_conceded_avg': homeTeam['conceded'].values[0], \
                 '1_conceded': row["away_goals"], \
                 'away_team_id': row['away_team_id'], \
                 '2_scored_avg': awayTeam['scored'].values[0], \
                 '2_scored': row["away_goals"], \
                 '2_conceded_avg': awayTeam['conceded'].values[0], \
                 '2_conceded': row["home_goals"], \
                 'total': row["home_goals"] + row["away_goals"]}
            dataSet1.at[index, :] = d
    

In [18]:
dataSet1 = dataSet1.sort_values(by=['date_time'])
dataSet1

,game_id,season,date_time,home_team_id,1_scored,1_conceded,1_scored_avg,1_conceded_avg,away_team_id,2_scored,2_conceded,2_scored_avg,2_conceded_avg,total
9192,2011020001,20112012,2011-10-06,6,1,2,2.93182,2.73864,4,2,1,3.1413,3.1087,3
11049,2011020002,20112012,2011-10-06,10,2,0,3,3.03659,8,0,2,2.81481,2.53086,2
10477,2011020005,20112012,2011-10-07,17,5,3,3.18824,2.27059,9,3,5,2.73864,3.06818,8
9869,2011020003,20112012,2011-10-07,23,3,4,2.75294,2.64706,5,4,3,3.36364,2.98864,7
10684,2011020009,20112012,2011-10-07,26,3,2,2.30208,2.14583,3,2,3,2.53465,2.33663,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429,2018030413,20182019,2019-06-02,19,2,7,2.78505,2.91589,6,7,2,2.97143,2.73333,9
11430,2018030414,20182019,2019-06-04,19,4,2,2.78505,2.91589,6,2,4,2.97143,2.73333,6
11431,2018030415,20182019,2019-06-07,6,1,2,2.97143,2.73333,19,2,1,2.78505,2.91589,3
11432,2018030416,20182019,2019-06-10,19,1,5,2.78505,2.91589,6,5,1,2.97143,2.73333,6


In [22]:
dataSetFinal = dataSet1.loc[:,['season','1_scored', '1_conceded','1_scored_avg', '1_conceded_avg', '2_scored', '2_conceded', '2_scored_avg', '2_conceded_avg', 'total']]

In [23]:
dataSetFinal 

,season,1_scored,1_conceded,1_scored_avg,1_conceded_avg,2_scored,2_conceded,2_scored_avg,2_conceded_avg,total
9192,20112012,1,2,2.93182,2.73864,2,1,3.1413,3.1087,3
11049,20112012,2,0,3,3.03659,0,2,2.81481,2.53086,2
10477,20112012,5,3,3.18824,2.27059,3,5,2.73864,3.06818,8
9869,20112012,3,4,2.75294,2.64706,4,3,3.36364,2.98864,7
10684,20112012,3,2,2.30208,2.14583,2,3,2.53465,2.33663,5
...,...,...,...,...,...,...,...,...,...,...
11429,20182019,2,7,2.78505,2.91589,7,2,2.97143,2.73333,9
11430,20182019,4,2,2.78505,2.91589,2,4,2.97143,2.73333,6
11431,20182019,1,2,2.97143,2.73333,2,1,2.78505,2.91589,3
11432,20182019,1,5,2.78505,2.91589,5,1,2.97143,2.73333,6


In [29]:
dataTest = dataSetFinal.loc[dataSetFinal.season == 20182019]

In [30]:
dataTest

,season,1_scored,1_conceded,1_scored_avg,1_conceded_avg,2_scored,2_conceded,2_scored_avg,2_conceded_avg,total
7617,20182019,7,0,3.39773,2.96591,0,7,2.97143,2.73333,7
7616,20182019,3,2,3.30682,3.14773,2,3,3.19753,2.7037,5
7624,20182019,3,4,3.67901,2.95062,4,3,3.58025,3.2716,7
7625,20182019,2,3,3.24691,2.91358,3,2,2.89011,3.02198,5
7622,20182019,1,2,3.04167,2.64583,2,1,2.62921,2.49438,3
...,...,...,...,...,...,...,...,...,...,...
11429,20182019,2,7,2.78505,2.91589,7,2,2.97143,2.73333,9
11430,20182019,4,2,2.78505,2.91589,2,4,2.97143,2.73333,6
11431,20182019,1,2,2.97143,2.73333,2,1,2.78505,2.91589,3
11432,20182019,1,5,2.78505,2.91589,5,1,2.97143,2.73333,6


In [31]:
dataTrain = dataSetFinal.loc[dataSetFinal.season != 20182019]

In [32]:
dataTrain

,season,1_scored,1_conceded,1_scored_avg,1_conceded_avg,2_scored,2_conceded,2_scored_avg,2_conceded_avg,total
9192,20112012,1,2,2.93182,2.73864,2,1,3.1413,3.1087,3
11049,20112012,2,0,3,3.03659,0,2,2.81481,2.53086,2
10477,20112012,5,3,3.18824,2.27059,3,5,2.73864,3.06818,8
9869,20112012,3,4,2.75294,2.64706,4,3,3.36364,2.98864,7
10684,20112012,3,2,2.30208,2.14583,2,3,2.53465,2.33663,5
...,...,...,...,...,...,...,...,...,...,...
633,20172018,2,4,3.23529,2.84314,4,2,3.14286,3.21429,6
634,20172018,2,4,3.23529,2.84314,4,2,3.14286,3.21429,6
635,20172018,3,2,3.14286,3.21429,2,3,3.23529,2.84314,5
636,20172018,3,0,3.23529,2.84314,0,3,3.14286,3.21429,3


In [44]:
dataSet1.to_csv('.\datasets\dataset1.csv', encoding='utf-8', index=False)

In [35]:
dataTest.to_csv('.\datasets\dataSet1Test.csv', encoding='utf-8', index=False)
dataTrain.to_csv('.\datasets\dataSet1Train.csv', encoding='utf-8', index=False)